# Workspace creation

_Valerio Ippolito - INFN Sezione di Roma_

In this tutorial, we will create a simple example workspace based on existing histogram input

We first import ROOT (we'll be using it interactively via pyROOT) and glob, a tool to conveniently loop over the content of a folder

In [ ]:
import ROOT
import glob

Let's identify the available files

In [ ]:
files = glob.glob('../data/*root')
print(files)

The structure of the file mimicks that of the histogram file created ("backed up") by workspace creation code. Histograms are named in an unique way, and stored in subfolders (of other subfolders), depending on which region (= statistically independent channel), sample (= physics process) and variation (= nominal, various systematic sources) they belong to.

Let's define a recursive function to unpack in an unbiased way this structure

In [ ]:
def list_file(f, spacing=''):
    for k in f.GetListOfKeys():        
        if k.GetClassName() == 'TDirectoryFile':
            print(f'{spacing} {k.GetName()}')
            list_file(k.ReadObj(), spacing=f'{spacing}\t')
        else: # we found a histo or similar
            print(f'{spacing} {k.GetName()} ({k.GetClassName()})')

... and another function to help us interpret the (long!) output. Here we expect a three-level structure:
1. first level is the region folder
2. second level is the sample name
3. third level is the variation name

In [ ]:
def find_samples(f):
    regions = set()
    samples = set()
    systs = set()
    for k in f.GetListOfKeys():
        if k.GetClassName() == 'TDirectoryFile':
            regions.add(k.GetName())
        for k2 in k.ReadObj().GetListOfKeys():
            if k2.GetClassName() == 'TDirectoryFile':
                samples.add(k2.GetName())
                for k3 in k2.ReadObj().GetListOfKeys():
                    if k3.GetClassName() == 'TDirectoryFile':
                        systs.add(k3.GetName())
    return regions, samples, systs

With these bullets in our gun, let's print out the content of the input files:

In [ ]:
fnames = {} # one per region
samples = {}
systs = {}

for fname in files:
    print(f'\nContent of file {fname}:')
    f = ROOT.TFile(fname)
    
    list_file(f)
    reg_, sam_, sys_ = find_samples(f)
    assert(len(reg_) == 1) # only one region per file
    fnames[list(reg_)[0]] = fname
    samples[list(reg_)[0]] = sam_
    systs[list(reg_)[0]] = sys_

A more convenient way to look at this output:

In [ ]:
for region in samples.keys():
    print(f'\nregion: {region}')
    print(f'samples: {samples[region]}')
    print(f'available variations: {systs[region]}')

Now that we understood what's available in the file, let's decide ourselves which subset of this info we will use to build our statistical analysis (our likelihood model).
We'll decide:
- which regions we want to consider (we may start with one of them and then extend to all of them)
- which set of histograms we want to take (the suffix `_regBin` is used for the final histograms, while there are other without this suffix we will ignore)
- how the data are called (we have a single set of data histograms, which contain the name `Data` - convenient, isn't it?)
- what's the name of the nominal variation (i.e. what's the folder containing the nominal histograms - surprisingly, we had called it `nominal` when creating the histograms)
- which samples we want to consider (TODO: a lists which so far isn't used anywhere, as we don't loop anymore on samples for educational reasons)
- which systematics we want to consider (TODO: remove, as that's a superset and there are also overall systs involved for some reason)

In [ ]:
regions = ['ljets_HThad_5j3b', 'ljets_Mbb_ge6jge4b']
k_suffix = '_regBin'
k_data = 'Data' # key for data
k_nominal = 'nominal' # key for nominal
samples = ['ttbar', 'singleTop', 'ttH']
systs = ['nominal', 'ttHXsec', 'BTag_B_NP1', 'BTag_B_NP2', 'BTag_B_NP3', 'ttXsec', 'JES_Scenario1_NP1', 'JES_Scenario1_NP2', 'BTag_C_NP1', 'tt_Shower', 'Lumi', 'stXsec', 'JER']

HistFactory is so nice with us that we are allowed to make many tests, provided we pay the price of deciding how to name them (and the corresponding outputs). File names will always contain some nickname (to which we conveniently add the prefix `ATLASIT`), followed by names depicting which channel or combination of channels these files refer to.

In [ ]:
nick = 'prova'

The goal of this notebook is to create a workspace, which means:
- a file containing the actual ROOT workspace, one per "configuration" of the likelihood model (single-channel, all-channels)
- a file containing the specification of how we built such workspace from the input ROOT files, in XML format

In [ ]:
!mkdir -p ../ws
!mkdir -p ../xml

Our likelihood model, and the meaning we give to it, is stored within a _measurement_ - an HistFactory concept which needs to know:
- how we want to nickname it
- where output files should be stored
- what's the parameter of interest of this measurement
- what are the parameters to be considered as a constant, if any - we typically include the default luminosity nuisance parameter created by HistFactory, called `Lumi`, within this "blacklist"
- what are the default settings of the default luminosity parameter, used by HistFactory whenever you specify that a channel should be normalized by luminosity (see `SetNormalizeByTheory`)

We are also nice people who like to decouple logical steps, so we ask HistFactory to kindly not do anything else than exporting the workspace into a ROOT file (i.e. please HistFactory do not perform any statistical analysis without our consent).

In [ ]:
meas = ROOT.RooStats.HistFactory.Measurement(f'ATLASIT_{nick}', f'ATLASIT_{nick}')
meas.SetOutputFilePrefix(f'../ws/ATLASIT_{nick}')

meas.SetPOI('mu_ttH')

meas.AddConstantParam('Lumi')
meas.SetLumi(1.0)
meas.SetLumiRelErr(0.0)
meas.SetExportOnly(True)

Shortly speaking, systematics are divided in:
- shape uncertainties, for which you normally provide a shape variation (i.e. another histogram which has the same binning but possibly different bin contents than the nominal one - for each region, channel and sample this systematic uncertainty applies to)
- normalisation-only uncertainties, which are provided in a simple form as "-1sigma" and "+1sigma" variations - where the meaning of the sign is given by you, the analyser, and not by any mathematical constraint (i.e. +1sigma variations may also reduce the normalisation of a histogram with respect to its nominal value).

Systematics are considered as correlated (i.e. they are represented by a single nuisance parameter) if they share the same name.

Let's address the boring task of writing down our normalisation-only uncertainties: we follow the totally reasonable practice of indicating first the -1sigma and then the +1sigma effect of each nuisance parameter on the normalisation of each histogram (and we need to be careful in remembering this order for the rest of our code):

In [ ]:
norm_systs = {
    'ljets_HThad_5j3b': {
        'ttH': {
            'lumi':   (0.95, 1.05),
            'ttXsec': (0.90, 1.10),
            'JES_Scenario1_NP1': (1.17705, 0.822951),
        },
        'ttbar': {
            'lumi':   (0.95, 1.05),
            'ttXsec': (0.94, 1.06),
            'JES_Scenario1_NP1': (0.926422, 1.07358),
        },
        'singleTop': {
            'lumi':   (0.95, 1.05),
            'stXsec': (0.95, 1.05),
            'JES_Scenario1_NP1': (0.892136, 1.0786),
        },
    },
     'ljets_Mbb_ge6jge4b': {
        'ttH': {
            'lumi':   (0.95, 1.05),
            'ttXsec': (0.90, 1.10),
            'JES_Scenario1_NP1': (0.946709, 1.05329),
        },
        'ttbar': {
            'lumi':   (0.95, 1.05),
            'ttXsec': (0.94, 1.06),
            'JES_Scenario1_NP1': (0.925296, 1.0747),
        },
        'singleTop': {
            'lumi':   (0.95, 1.05),
            'stXsec': (0.95, 1.05),
            'JES_Scenario1_NP1': (0.898541, 1.10146),
        },
    },
}

We then follow this logic:
1. we first create a channel (corresponding to some set of statistically-independent data)
2. we tell HistFactory where (meaning: in which file, under which subdirectory path and more specifically in which histogram) to find the data for this channel
3. we may indulge in specifying how uncertainties related to the limited MC statistics in signal/background histograms should be dealt with, in this channel
4. we then add the samples which contribute to this channel, specifying where to find their nominal histograms, and which normalisation-only (`AddOverallSys`) and also-shape uncertainties (`AddHistoSys`) should be considered (keeping in mind that variations of any kind which share the same name are correlated)
5. we also add free parameters to fit for determining the normalisation of our signal (and sometimes background) samples
6. we add each sample to the channel
7. we add each channel to the measurement, and go back to 1. until all channels were considered

In [ ]:
for region in regions:
    chan = ROOT.RooStats.HistFactory.Channel(region)

    # add data
    chan.SetData(f'{region}_{k_data}{k_suffix}', # histo name, histo file, histo path
                 fnames[region],
                 f'{region}/{k_data}/{k_nominal}')
    
    chan.SetStatErrorConfig(0.05, 'Poisson')

    ########
    # add signal
    sname = 'ttH'
    sample_ttH = ROOT.RooStats.HistFactory.Sample(sname, # name, histo name, histo file, histo path
                                              f'{region}_{sname}{k_suffix}',
                                              fnames[region],
                                              f'{region}/{sname}/{k_nominal}')
    sample_ttH.SetNormalizeByTheory(False) # we use our own NP for luminosity
    
    for sysname, sysvalue in norm_systs[region][sname].items():
        sample_ttH.AddOverallSys(sysname, sysvalue[0], sysvalue[1]) # name, -1sigma effect, +1sigma effect
    sample_ttH.AddNormFactor('mu_ttH', 0, -10, 20) # NB default value is zero
    sample_ttH.GetStatError().Activate(False)
    chan.AddSample(sample_ttH)
    
    ########
    # add ttbar
    sname = 'ttbar'
    sample_tt = ROOT.RooStats.HistFactory.Sample(sname, # name, histo name, histo file, histo path
                                              f'{region}_{sname}{k_suffix}',
                                              fnames[region],
                                              f'{region}/{sname}/{k_nominal}')
    sample_tt.SetNormalizeByTheory(False) # we use our own NP for luminosity
    
    for sysname, sysvalue in norm_systs[region][sname].items():
        sample_tt.AddOverallSys(sysname, sysvalue[0], sysvalue[1]) # name, -1sigma effect, +1sigma effect
    
    syst_ = 'JES_Scenario1_NP1'
    sample_tt.AddHistoSys(syst_, # name, histo name -1sigma, histo file -1sigma, histo path -1sigma, histo name +1sigma, histo file +1sigma, histo path +1sigma
                          f'{region}_{sname}_{syst_}_Shape_Down{k_suffix}', fnames[region], f'{region}/{sname}/{syst_}',
                          f'{region}_{sname}_{syst_}_Shape_Up{k_suffix}', fnames[region], f'{region}/{sname}/{syst_}',
                         ) 
    sample_tt.AddNormFactor('mu_tt', 1, 0, 20)
    sample_tt.GetStatError().Activate(False)
    chan.AddSample(sample_tt)
    
    ########
    # add singleTop
    sname = 'singleTop'
    sample_st = ROOT.RooStats.HistFactory.Sample(sname, # name, histo name, histo file, histo path
                                              f'{region}_{sname}{k_suffix}',
                                              fnames[region],
                                              f'{region}/{sname}/{k_nominal}')
    sample_st.SetNormalizeByTheory(False) # we use our own NP for luminosity
    
    for sysname, sysvalue in norm_systs[region][sname].items():
        sample_st.AddOverallSys(sysname, sysvalue[0], sysvalue[1]) # name, -1sigma effect, +1sigma effect
    
    sample_st.GetStatError().Activate(False)
    chan.AddSample(sample_st)


    
    # finally, collect the channel
    meas.AddChannel(chan)

Eventually, we ask HistFactory to actually go and check the histograms, do its magic and create The Likelihood Model. We also persist this likelihood model in XML format, for our afternoons of debugging.

In [ ]:
meas.CollectHistograms()
meas.PrintTree()
meas.PrintXML('../xml', meas.GetOutputFilePrefix())

ROOT.RooStats.HistFactory.MakeModelAndMeasurementFast(meas)

And that's it!

In [ ]:
!ls ../ws

In [ ]:
!ls ../xml

## Exercise

Let's try to add more systematics (all those we listed before and for which we have histogram inputs in the root files).
Specifically, we want to have all these systematics added: `'BTag_B_NP1', 'BTag_B_NP2', 'BTag_B_NP3', 'JES_Scenario1_NP2', 'BTag_C_NP1', 'tt_Shower', 'JER'` (on top of `'JES_Scenario1_NP1'` that we added already).
All of these have both a "normalization" and "shape" effects.
For the normalization effects, we need to add by hand the numbers, as we did for `lumi`, `JES_Scenario1_NP1` and the cross-section uncertainties.

These are the numbers to add:
```
Region ljets_HThad_5j3b:
ttH:
  "BTag_B_NP1"  High="0.8697"  Low="1.1303"
  "BTag_B_NP2"  High="1.02549"  Low="0.974511"
  "BTag_B_NP3"  High="0.982417"  Low="1.01758"
ttbar:
  "BTag_B_NP1"  High="0.87674"  Low="1.12326"
  "BTag_B_NP2"  High="1.02837"  Low="0.971628"
  "BTag_B_NP3"  High="0.986622"  Low="1.01338"
  "BTag_C_NP1"  High="0.916677"  Low="1.08332"
  "tt_Shower"  High="1.06324"  Low="0.936763"
singleTop:
  "BTag_B_NP1"  High="0.874054"  Low="1.12595"
  "BTag_B_NP2"  High="1.0217"  Low="0.978299"
  "BTag_B_NP3"  High="0.98987"  Low="1.01013"
  "BTag_C_NP1"  High="0.920117"  Low="1.07988"
  "JES_Scenario1_NP2"  High="1.034"  Low="0.965996"
  
Region ljets_Mbb_ge6jge4b:
ttH:
  "BTag_B_NP1"  High="0.780949"  Low="1.21905"
  "BTag_B_NP2"  High="1.04522"  Low="0.954776"
  "BTag_B_NP3"  High="0.976712"  Low="1.02329"
ttbar:
  "BTag_B_NP1"  High="0.874133"  Low="1.12587"
  "BTag_B_NP2"  High="1.02803"  Low="0.971967"
  "BTag_B_NP3"  High="0.986524"  Low="1.01348"
  "BTag_C_NP1"  High="0.915986"  Low="1.08401"
  "tt_Shower"  High="1.06498"  Low="0.935018"
singleTop:
  "BTag_B_NP1"  High="0.870235"  Low="1.12977"
  "BTag_B_NP2"  High="1.02002"  Low="0.97998"
  "BTag_B_NP3"  High="0.989511"  Low="1.01049"
  "BTag_C_NP1"  High="0.920832"  Low="1.07917"
  "JES_Scenario1_NP2"  High="1.03127"  Low="0.968725"
```
(where nothing is listed, the normalization effect is zero).

Then, for the shape part, we need to replicate these lines:
```
    syst_ = 'JES_Scenario1_NP1'
    sample_tt.AddHistoSys(syst_, # name, histo name -1sigma, histo file -1sigma, histo path -1sigma, histo name +1sigma, histo file +1sigma, histo path +1sigma
                          f'{region}_{sname}_{syst_}_Shape_Down{k_suffix}', fnames[region], f'{region}/{sname}/{syst_}',
                          f'{region}_{sname}_{syst_}_Shape_Up{k_suffix}', fnames[region], f'{region}/{sname}/{syst_}',
                         )
```
e.g. by replacing them with something like this (in the case of the ttbar background):
```
    shapeSysts = ['BTag_B_NP1','BTag_B_NP2','BTag_B_NP3','BTag_C_NP1','tt_Shower','JER']
    for syst_ in shapeSysts:
        sample_tt.AddHistoSys(syst_, 
                              f'{region}_{sname}_{syst_}_Shape_Down{k_suffix}', fnames[region], f'{region}/{sname}/{syst_}',
                              f'{region}_{sname}_{syst_}_Shape_Up{k_suffix}', fnames[region], f'{region}/{sname}/{syst_}',
                             )
```
For the other samples, the lists of shape systematics are the following:
```
ttH: ['BTag_B_NP1','BTag_B_NP2','BTag_B_NP3','BTag_C_NP1']
singleTop: none
```